In [1]:
using LinearAlgebra
using SparseArrays
using Printf

In [2]:
import MathProgBase
MPB = MathProgBase

import Mosek
import Mosek:MosekSolver

MPB.getbarrieriter(m::Mosek.MosekMathProgSolverInterface.MosekLinearQuadraticModel) = Mosek.getintinf(m.task, Mosek.MSK_IINF_INTPNT_ITER)
MPB.getsimplexiter(m::Mosek.MosekMathProgSolverInterface.MosekLinearQuadraticModel) = Mosek.getintinf(m.task,Mosek.MSK_IINF_SIM_PRIMAL_ITER)+Mosek.getintinf(m.task,Mosek.MSK_IINF_SIM_DUAL_ITER)

In [3]:
solver = MosekSolver(MSK_IPAR_LOG=0);

In [4]:
import Linda

┌ Info: Recompiling stale cache file /home/mathieu/.julia/compiled/v1.0/Linda/hN7lX.ji for Linda [4f5ebc0c-70dc-5a88-8eac-7b9fb65244d9]
└ @ Base loading.jl:1187


# TSSP formulation

Deterministic equivalent:

\begin{align*}
    (DEP) \ \ \ \min_{x, y} \ \ \ & c^{T}x + \sum_{i} p_{i} q_{i}^{T} y_{i}\\
    s.t. \ \ \ 
    & Ax = b,\\
    & T_{i} x + W_{i}y_{i} = h_{i}, \ \ \ \forall i,\\
    & x \geq 0,\\
    & y_{i} \geq 0, \ \ \ \forall i.
\end{align*}

Dual of (DEP) writes:
\begin{align*}
    (D) \ \ \ \max_{\eta, \theta} \ \ \ & b^{T}\eta + \sum_{i} h_{i}^{T} \theta_{i}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i} T_{i}^{T} \theta_{i} \leq c,\\
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}, \ \ \ \forall i.
\end{align*}

We wolve (D) by column generation. Note that it amounts to the same as solving (DEP) by Benders.

The Master problem writes
\begin{align*}
    (D) \ \ \ \min_{\eta, \theta} \ \ \ & -b^{T}\eta - \sum_{i, \omega} h_{i, \omega} \lambda_{i, \omega} - \sum_{i, \rho} h_{i, \rho} \lambda_{i, \rho}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i, \omega} t_{i, \omega} \lambda_{i, \omega} + \sum_{i, \rho} t_{i, \rho} \lambda_{i, \rho} \leq c, & [\pi]\\
    & \sum_{i} \lambda_{i, \omega} = 1, \ \ \ \forall i, & [\sigma_{i}]\\
    & \lambda \geq 0.
\end{align*}

and the sub-problem writes:
\begin{align*}
    (SP) \ \ \ \min_{\theta_{i}} \ \ \ & (-h_{i}^{T} - \pi^{T} T_{i}^{T}) \theta_{i} - \sigma_{i}\\
    s.t. \ \ \ 
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}.
\end{align*}

# Source code

In [5]:
include("src/TSSP/parser/core_parser.jl")
include("src/TSSP/parser/time_parser.jl")
include("src/TSSP/parser/stoch_parser.jl")

parse_block_line! (generic function with 1 method)

In [6]:
function generate_scenario_data(
        A_ref, T_ref, W_ref, q_ref, h_ref,
        row2idx, col2idx, 
        prob, coeffs
)
    m1, n1 = size(A_ref)
    m2, n2 = size(W_ref)
    m2 == size(T_ref, 1) || error("T and W has different number of rows.")
    n1 == size(T_ref, 2) || error("T and A has different number of columns.")
    
    # Create problem data
    h_ = copy(h_ref)          # Right-hand side
    q_ = copy(q_ref)  # Objective ()
    
    T_ = copy(T_ref)          # T_r matrix
    W_ = copy(W_ref)          # W_r
    
    
    for (k, v) in coeffs
        row::Int = row2idx[k[1]]
        col::Int = col2idx[k[2]]
        
        row == 0 || row > m1 || error("Entry given for first-stage row $(k[1])")
        
        # Check four cases
        if row == 0 && col > 0
            # Objective coefficient
#             println("Obj: $k: $v")
            q_[col - n1] = v
            
        elseif row > 0 && col == 0
            # Right-hand side
#             println("Rhs: $k \t $v")
            h_[row - m1] = v
            
        elseif row > 0 && col > 0
            # Check whether belongs to T or W
            if col <= n1
                T_[row - m1, col] = v
            else
                W_[row - m1, col - n1] = v
            end
        else
            eror("Wrong combination of row/col indices: $k")
        end
    end
    
    q_ .*= prob
    
    return T_, W_, q_, h_
end

generate_scenario_data (generic function with 1 method)

# Test instance files

In [64]:
f_cor = "data/tssp/4node.cor"
f_sto = "data/tssp/4node.sto.1024"
f_tim = "data/tssp/4node.tim"

"data/tssp/4node.tim"

# Parse test instance

## Parse `.cor` file

In [65]:
# Extract data from .cor file
d, row2idx, col2idx, I, J, V, objI, objV, rhsI, rhsV, senses = parse_cor_file(f_cor);

## Parse `.tim` file

In [66]:
name, is_lp, col_periods, row_periods = parse_time_file(f_tim);

In [67]:
# Extract indices of first/second period rows/columns
col1, col2 = (
    col2idx[col_periods[1]]:(col2idx[col_periods[2]]-1),  # First-period variables
    col2idx[col_periods[2]]:d["ncols"]  # Second-period variables
)
row1, row2 = (
    row2idx[row_periods[1]]:(row2idx[row_periods[2]]-1),  # First-period rows
    row2idx[row_periods[2]]:d["nrows"]  # Second-period variables
)

if row1[1] == 0
    row1 = 1:row1[end]
end
if col1[1] == 0
    col1 = 1:col1[end]
end

In [68]:
# Extract vector of objective coefficients
obj = sparsevec(objI, objV, d["ncols"])
c, q = Vector(obj[col1]), Vector(obj[col2])

([50.0, 65.0, 45.0, 70.0, 60.0, 55.0, 50.0, 60.0, 55.0, 40.0  …  76.8, 57.6, 62.4, 67.2, 57.6, 38.4, 43.2, 57.6, 43.2, 62.4], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [69]:
# Extract right-hand side vectors
rhs = sparsevec(rhsI, rhsV, d["nrows"])
b, h = Vector(rhs[row1]), Vector(rhs[row2])

([25.0, 25.0, 25.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 480.0, 240.0, 0.0, 0.0], [5.0, 8.0, 4.0, 4.5, 6.7, 4.2, 10.1, 8.3, 12.2, 3.2  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [70]:
# Extract constraint matrices
A_ = sparse(I, J, V);

A = A_[row1, col1]
T = A_[row2, col1]
W = A_[row2, col2];

### Convert `A` to standard form

In [71]:
n_ctr = sum(senses[row1] .!= "E")  # Number of slack variables to be added
println("n_ctr = $n_ctr")
m1 = length(row1)
m2 = length(row2)

n1 = length(col1)
n2 = length(col2)

S_mat = sparse(
    [i for i in 1:m1 if senses[i] != "E"],
    collect(1:n_ctr),
    [senses[i] == "L" ? 1.0 : -1.0 for i in 1:m1 if senses[i] != "E"],
    m1, n_ctr
);

n_ctr = 8


In [72]:
# Add slack coefficients to matrix
A = hcat(A, S_mat);
T = hcat(T, spzeros(m2, n_ctr))

# Add zero coeffs to objective vector
append!(c, zeros(n_ctr))

# Shift all second-stage variables to account for first-stage slack
for (k, v) in col2idx
    if v > n1
        col2idx[k] += n_ctr
    end
end
col1 = col1[1]:(m1+n_ctr)

1:22

In [73]:
senses[row1] .= "E";

### Convert W to standard form

In [74]:
# Number of non-equality constraints
n_ctr = sum(senses[row2] .!= "E")  # number of slack variables to be added

m2 = length(row2)
m1 = length(row1)

S_mat = sparse(
    [i for i in 1:m2 if senses[m1+i] != "E"],
    collect(1:n_ctr),
    [senses[m1+i] == "L" ? 1.0 : -1.0 for i in 1:m2 if senses[m1+i] != "E"],
    m2, n_ctr
);

In [75]:
# Add slack variables to W matrix
W = hcat(W, S_mat)

# Add slack variables to objective
append!(q, zeros(n_ctr));

In [76]:
senses[row2] .= "E";

In [77]:
# row2 = (m1+1):(m1+m2+n_ctr)

## Parse `.sto` file

In [78]:
name, indeps, blocks = parse_sto_file(f_sto);

In [79]:
# Extract each scenario from .sto data
S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
S = [prod(collect(s)) for s in S_]
length(S)

1024

# Construct deterministic equivalent problem

In [80]:
@time pb_data = [generate_scenario_data(A, T, W, q, h, row2idx, col2idx, s.p, s.values) for s in S][:];

  0.200343 seconds (1.02 M allocations: 61.901 MiB, 10.82% gc time)


In [81]:
T_ = [x[1] for x in pb_data];
W_ = [x[2] for x in pb_data];
q_ = [x[3] for x in pb_data];
h_ = [x[4] for x in pb_data];

In [82]:
m1, n1 = size(A)
m2, n2 = size(W)
R = length(S)

1024

In [60]:
@time U = [
    hcat(
        T_[r],
        spzeros(m2, n2*(r-1)),
        W_[r],
        spzeros(m2, n2*(R-r))
    )
    for r in 1:R
];


@time vU = vcat(U...);

  0.089284 seconds (88.33 k allocations: 130.592 MiB, 22.83% gc time)
  0.492280 seconds (10.50 M allocations: 169.217 MiB, 11.05% gc time)


In [61]:
rhs_ = vcat(b, h_...)
obj_ = vcat(c, q_...)
senses_ = vcat(senses[1:m1], repeat(senses[(m1+1):end], R))
n_ = length(obj_)
m_ = length(rhs_)
@time A_ = vcat(
    hcat(A,spzeros(m1, n2*R)),
    vU
);

rhs_l = rhs_ .- Inf .* (senses_ .== "L");
rhs_u = rhs_ .+ Inf .* (senses_ .== "G");

  0.000201 seconds (31 allocations: 551.516 KiB)


In [62]:
size(A_), nnz(A_)

((4058, 8118), 18934)

In [63]:
airl = MPB.LinearQuadraticModel(solver)
MPB.loadproblem!(airl, A_, zeros(n_), Inf*ones(n_), obj_, rhs_l, rhs_u, :Min);

In [64]:
MPB.optimize!(airl)

In [65]:
MPB.getobjval(airl)

250262.04979946063

# Column-generation formulation

## Create oracles

In [98]:
# Instanciate sub-problems
pool = Linda.Oracle.LindaOraclePool([
    Linda.Oracle.LindaOracleMIP(
        r,
        -h_[r],
        transpose(T_[r]),
        transpose(W_[r]),
        -Inf*ones(length(q_[r])), q_[r],  # right-hand side
        fill(:Cont, length(h_[r])),
        -Inf*ones(length(h_[r])),
        Inf*ones(length(h_[r])),
        solver
    )
    for r in 1:R
]);

In [99]:
# Create initial columns by computing minimum cost schedule for each house
initial_cols = Linda.Column[]

@time for o in pool.oracles
#     Linda.Oracle.query!(o, zeros(size(T, 2)), Inf)
#     col = Linda.Oracle.get_new_columns(o)[1]
    col = Linda.Column(1e3, zeros(size(T, 2)), true, o.index)
    push!(initial_cols, col)
end
;

  0.000759 seconds (4.62 k allocations: 696.453 KiB)


### Create Master

In [100]:
function generate_rmp(A, R, rowlb, rowub, obj, columns, rmp_solver)
    
    rmp = MPB.LinearQuadraticModel(rmp_solver)
    
    A0 = vcat(
        spzeros(R, 2*size(A, 2)+size(A, 1)),
        hcat(A, -A, sparse(1:size(A, 1), 1:size(A, 1), ones(size(A, 1))))
    )
    A_ = hcat(
        A0,
        vcat(
            hcat([sparsevec([c.idx_subproblem], [1.0], R) for c in columns]...),
            hcat([c.col for c in columns]...)
        )
    )
    
    obj_ = vcat(obj, -obj, zeros(size(A, 1)), vcat([col.cost for col in columns]...))
    
    rowlb_ = vcat(ones(R), rowlb)
    rowub_ = vcat(ones(R), rowub)
    
#     display([rowlb_ rowub_])
    
    collb_ = zeros(2*length(obj) + size(A, 1) + length(columns))
    colub_ = Inf .* ones(2*length(obj) + size(A, 1) + length(columns))
    
#     println(size(A_))
#     println(length(collb_), "\t", length(colub_))
    
    MPB.loadproblem!(rmp, A_, collb_, colub_, obj_, rowlb_, rowub_, :Min)
    
    return rmp
end

generate_rmp (generic function with 1 method)

In [101]:
rmp = generate_rmp(transpose(A), R, c, c, -b, initial_cols, 
    MosekSolver(
        MSK_IPAR_PRESOLVE_USE=0,
        MSK_IPAR_LOG=0, 
#         SK_IPAR_OPTIMIZER=3
    )
)

mp = Linda.LindaMaster(R, length(c), c, 2*size(A, 1)+size(A, 2), initial_cols, rmp);

In [102]:
env = Linda.LindaEnv();
env[:verbose] = 1
env[:num_cgiter_max] = 100
env[:num_columns_max] = 1+div(R, 10)

103

### Solve

In [103]:
Linda.solve_colgen!(env, mp, pool)

Itn     Primal Obj        Dual Obj         NCols    MP(s)    SP(s)   Tot(s)  BarIter  SpxIter
   0    +1.0240000e+06            -Inf      1024     0.01     0.05     0.06        7        0
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1112) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1113) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1114) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1115) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1116) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1117) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1118) of matrix 'A'.
MOSEK warning 710: #8 (nearly) zero elements are specified in sparse col '' (1119) of matrix 'A'.
MOSEK warning 710: #8 (nearl

InterruptException: InterruptException:

In [51]:
Mosek.putintparam(rmp.task, Mosek.MSK_IPAR_LOG, 1)
Mosek.putintparam(rmp.task, Mosek.MSK_IPAR_OPTIMIZER, 3)

In [52]:
MPB.optimize!(rmp)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 37513           
  Cones                  : 0               
  Scalar variables       : 55523           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Optimizer terminated. Time: 0.03    


Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: 1.9500000333e+10    nrm: 3e+02    Viol.  con: 1e-03    var: 0e+00  
  Dual.    obj: 1.9500000334e+10    nrm: 1e+06    Viol.  con: 0e+00    var: 2e-07  

Basic solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 1.9500000334e+10    nrm: 1e+00    Viol.  con: 1e-16    var: 0e+00  
  Dual.    obj: 1.9500000334e+10    nrm: 1e+06    Viol.  con: 0e+00    var: 3e-14  


In [78]:
MPB.getobjval(airl)

250262.04979946063